In [69]:
import os
from typing import List
from typing import Tuple
import logging
from collections import defaultdict
from collections import Counter
import json
import torch
import numpy as np

import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..', 'dataset'))
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter
if isnotebook():
    device = torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

FORMAT = "%(asctime)-15s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO,
                    datefmt="%Y-%m-%d %H:%M")
logger = logging.getLogger(__name__)

from ReaSCAN_dataset import *

In [37]:
path_to_data = "../../../data-files/ReaSCAN-compositional/data-compositional-splits.txt"
logger.info(f"Reading dataset from file: {path_to_data}...")
data_json = json.load(open(path_to_data, "r"))

2021-05-17 15:44 Reading dataset from file: ../../../data-files/ReaSCAN-compositional/data-compositional-splits.txt...


In [61]:
train_dataset = ReaSCANDataset(
    data_json=data_json, 
    save_directory="../../../data-files/ReaSCAN-compositional/", 
    input_vocabulary_file="input_vocabulary.txt",
    target_vocabulary_file="target_vocabulary.txt",
    k=100, 
    split="train", 
    generate_vocabulary=False
)
train_dataset.read_dataset(
    max_examples=100,
    simple_situation_representation=True
)

2021-05-17 17:17 Formulating the dataset from the passed in json file...
2021-05-17 17:17 Loading vocabularies...
2021-05-17 17:17 Done loading vocabularies.
2021-05-17 17:17 Converting dataset to tensors...
2021-05-17 17:17 Loading few examples into the training set for few-shots learning...
2021-05-17 17:17 The following idx examples are selected for few-shot learning:
2021-05-17 17:17 [5010, 10723, 6059, 26887, 32423, 19034, 4813, 22516, 47822, 39602, 36936, 18548, 11016, 34759, 29562, 45122, 45528, 37179, 14605, 20723, 6153, 8403, 19641, 27385, 16961, 11031, 3893, 41565, 47631, 6607, 35161, 3186, 44075, 38587, 35049, 827, 26706, 24965, 1519, 33543, 34917, 22927, 176, 15244, 35676, 23533, 28085, 445, 39335, 44803, 27619, 12948, 31673, 838, 19363, 19006, 25030, 48495, 16274, 49237, 11232, 31648, 14557, 34082, 13098, 23645, 36752, 43737, 16789, 16065, 4473, 31989, 34094, 28212, 21897, 13052, 20681, 15528, 47544, 37475, 20677, 32127, 20403, 13209, 27369, 29267, 8333, 7897, 33133, 40933

In [63]:
test_dataset = ReaSCANDataset(
    data_json=data_json, 
    save_directory="../../../data-files/ReaSCAN-compositional/", 
    input_vocabulary_file="input_vocabulary.txt",
    target_vocabulary_file="target_vocabulary.txt",
    k=0, 
    split="dev", 
    generate_vocabulary=False
)
test_dataset.read_dataset(
    max_examples=100,
    simple_situation_representation=True
)

2021-05-17 17:21 Formulating the dataset from the passed in json file...
2021-05-17 17:21 Loading vocabularies...
2021-05-17 17:21 Done loading vocabularies.
2021-05-17 17:21 Converting dataset to tensors...


In [68]:
import logging
import torch
import os
from torch.optim.lr_scheduler import LambdaLR
import random
import numpy as np

import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

from seq2seq.model import Model
from seq2seq.ReaSCAN_dataset import *
from seq2seq.helpers import log_parameters
from seq2seq.evaluate import evaluate

logger = logging.getLogger(__name__)
use_cuda = True if torch.cuda.is_available() else False


def train(
    data_path: str, 
    data_directory: str, 
    generate_vocabularies: bool, 
    input_vocab_path: str,   
    target_vocab_path: str, 
    embedding_dimension: int, 
    num_encoder_layers: int, 
    encoder_dropout_p: float,
    encoder_bidirectional: bool, 
    training_batch_size: int, 
    test_batch_size: int, 
    max_decoding_steps: int,
    num_decoder_layers: int, 
    decoder_dropout_p: float, 
    cnn_kernel_size: int, 
    cnn_dropout_p: float,
    cnn_hidden_num_channels: int, 
    simple_situation_representation: bool, 
    decoder_hidden_size: int,
    encoder_hidden_size: int, 
    learning_rate: float, 
    adam_beta_1: float, 
    adam_beta_2: float, 
    lr_decay: float,
    lr_decay_steps: int, 
    resume_from_file: str, 
    max_training_iterations: int, 
    output_directory: str,
    print_every: int, 
    evaluate_every: int, 
    conditional_attention: bool, 
    auxiliary_task: bool,
    weight_target_loss: float, 
    attention_type: str, 
    k: int, 
    max_training_examples=None, 
    seed=42, **kwargs
):
    if isnotebook():
        device = torch.device("cpu")
    else:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cfg = locals().copy()

    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    logger.info("Loading all data into memory...")
    logger.info(f"Reading dataset from file: {data_path}...")
    data_json = json.load(open(data_path, "r"))
    
    logger.info("Loading Training set...")
    training_set = ReaSCANDataset(
        data_json, data_directory, split="train",
        input_vocabulary_file=input_vocab_path,
        target_vocabulary_file=target_vocab_path,
        generate_vocabulary=generate_vocabularies, k=k
    )
    training_set.read_dataset(
        max_examples=max_training_examples,
        simple_situation_representation=simple_situation_representation
    )
    logger.info("Done Loading Training set.")
    logger.info("  Loaded {} training examples.".format(training_set.num_examples))
    logger.info("  Input vocabulary size training set: {}".format(training_set.input_vocabulary_size))
    logger.info("  Most common input words: {}".format(training_set.input_vocabulary.most_common(5)))
    logger.info("  Output vocabulary size training set: {}".format(training_set.target_vocabulary_size))
    logger.info("  Most common target words: {}".format(training_set.target_vocabulary.most_common(5)))

    if generate_vocabularies:
        training_set.save_vocabularies(input_vocab_path, target_vocab_path)
        logger.info("Saved vocabularies to {} for input and {} for target.".format(input_vocab_path, target_vocab_path))

    logger.info("Loading Dev. set...")
    test_set = ReaSCANDataset(
        data_json, data_directory, split="dev",
        input_vocabulary_file=input_vocab_path,
        target_vocabulary_file=target_vocab_path,
        generate_vocabulary=generate_vocabularies, k=0
    )
    test_set.read_dataset(
        max_examples=None,
        simple_situation_representation=simple_situation_representation
    )

    # Shuffle the test set to make sure that if we only evaluate max_testing_examples we get a random part of the set.
    test_set.shuffle_data()
    logger.info("Done Loading Dev. set.")

    return
    
    
#     model = Model(input_vocabulary_size=training_set.input_vocabulary_size,
#                   target_vocabulary_size=training_set.target_vocabulary_size,
#                   num_cnn_channels=training_set.image_channels,
#                   input_padding_idx=training_set.input_vocabulary.pad_idx,
#                   target_pad_idx=training_set.target_vocabulary.pad_idx,
#                   target_eos_idx=training_set.target_vocabulary.eos_idx,
#                   **cfg)
#     model = model.cuda() if use_cuda else model
#     log_parameters(model)
#     trainable_parameters = [parameter for parameter in model.parameters() if parameter.requires_grad]
#     optimizer = torch.optim.Adam(trainable_parameters, lr=learning_rate, betas=(adam_beta_1, adam_beta_2))
#     scheduler = LambdaLR(optimizer,
#                          lr_lambda=lambda t: lr_decay ** (t / lr_decay_steps))

#     # Load model and vocabularies if resuming.
#     start_iteration = 1
#     best_iteration = 1
#     best_accuracy = 0
#     best_exact_match = 0
#     best_loss = float('inf')
#     if resume_from_file:
#         assert os.path.isfile(resume_from_file), "No checkpoint found at {}".format(resume_from_file)
#         logger.info("Loading checkpoint from file at '{}'".format(resume_from_file))
#         optimizer_state_dict = model.load_model(resume_from_file)
#         optimizer.load_state_dict(optimizer_state_dict)
#         start_iteration = model.trained_iterations
#         logger.info("Loaded checkpoint '{}' (iter {})".format(resume_from_file, start_iteration))

#     logger.info("Training starts..")
#     training_iteration = start_iteration
#     while training_iteration < max_training_iterations:

#         # Shuffle the dataset and loop over it.
#         training_set.shuffle_data()
#         for (input_batch, input_lengths, _, situation_batch, _, target_batch,
#              target_lengths, agent_positions, target_positions) in training_set.get_data_iterator(
#                 batch_size=training_batch_size):
#             is_best = False
#             model.train()

#             # Forward pass.
#             target_scores, target_position_scores = model(commands_input=input_batch, commands_lengths=input_lengths,
#                                                           situations_input=situation_batch, target_batch=target_batch,
#                                                           target_lengths=target_lengths)
#             loss = model.get_loss(target_scores, target_batch)
#             if auxiliary_task:
#                 target_loss = model.get_auxiliary_loss(target_position_scores, target_positions)
#             else:
#                 target_loss = 0
#             loss += weight_target_loss * target_loss

#             # Backward pass and update model parameters.
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()
#             model.update_state(is_best=is_best)

#             # Print current metrics.
#             if training_iteration % print_every == 0:
#                 accuracy, exact_match = model.get_metrics(target_scores, target_batch)
#                 if auxiliary_task:
#                     auxiliary_accuracy_target = model.get_auxiliary_accuracy(target_position_scores, target_positions)
#                 else:
#                     auxiliary_accuracy_target = 0.
#                 learning_rate = scheduler.get_lr()[0]
#                 logger.info("Iteration %08d, loss %8.4f, accuracy %5.2f, exact match %5.2f, learning_rate %.5f,"
#                             " aux. accuracy target pos %5.2f" % (training_iteration, loss, accuracy, exact_match,
#                                                                  learning_rate, auxiliary_accuracy_target))

#             # Evaluate on test set.
#             if training_iteration % evaluate_every == 0:
#                 with torch.no_grad():
#                     model.eval()
#                     logger.info("Evaluating..")
#                     accuracy, exact_match, target_accuracy = evaluate(
#                         test_set.get_data_iterator(batch_size=1), model=model,
#                         max_decoding_steps=max_decoding_steps, pad_idx=test_set.target_vocabulary.pad_idx,
#                         sos_idx=test_set.target_vocabulary.sos_idx,
#                         eos_idx=test_set.target_vocabulary.eos_idx,
#                         max_examples_to_evaluate=kwargs["max_testing_examples"])
#                     logger.info("  Evaluation Accuracy: %5.2f Exact Match: %5.2f "
#                                 " Target Accuracy: %5.2f" % (accuracy, exact_match, target_accuracy))
#                     if exact_match > best_exact_match:
#                         is_best = True
#                         best_accuracy = accuracy
#                         best_exact_match = exact_match
#                         model.update_state(accuracy=accuracy, exact_match=exact_match, is_best=is_best)
#                     file_name = "checkpoint.pth.tar".format(str(training_iteration))
#                     if is_best:
#                         model.save_checkpoint(file_name=file_name, is_best=is_best,
#                                               optimizer_state_dict=optimizer.state_dict())

#             training_iteration += 1
#             if training_iteration > max_training_iterations:
#                 break
#     logger.info("Finished training.")


In [80]:
import argparse
import logging
import os
import torch

from seq2seq.ReaSCAN_dataset import *
from seq2seq.model import Model
from seq2seq.train import train
from seq2seq.predict import predict_and_save

FORMAT = "%(asctime)-15s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.DEBUG,
                    datefmt="%Y-%m-%d %H:%M")
logger = logging.getLogger(__name__)
if isnotebook():
    use_cuda = False
else:
    use_cuda = True if torch.cuda.is_available() else False


if use_cuda:
    logger.info("Using CUDA.")
    logger.info("Cuda version: {}".format(torch.version.cuda))

def arg_parse():
    parser = argparse.ArgumentParser(description="Sequence to sequence models for ReaSCAN")

    # General arguments
    parser.add_argument("--mode", type=str, default="run_tests", help="train, test or predict", required=True)
    parser.add_argument("--output_directory", type=str, default="output", help="In this directory the models will be "
                                                                               "saved. Will be created if doesn't exist.")
    parser.add_argument("--resume_from_file", type=str, default="", help="Full path to previously saved model to load.")

    # Data arguments
    parser.add_argument("--split", type=str, default="test", help="Which split to get from Grounded Scan.")
    parser.add_argument("--data_directory", type=str, default="data/uniform_dataset", help="Path to folder with data.")
    parser.add_argument("--input_vocab_path", type=str, default="training_input_vocab.txt",
                        help="Path to file with input vocabulary as saved by Vocabulary class in gSCAN_dataset.py")
    parser.add_argument("--target_vocab_path", type=str, default="training_target_vocab.txt",
                        help="Path to file with target vocabulary as saved by Vocabulary class in gSCAN_dataset.py")
    parser.add_argument("--generate_vocabularies", dest="generate_vocabularies", default=False, action="store_true",
                        help="Whether to generate vocabularies based on the data.")
    parser.add_argument("--load_vocabularies", dest="generate_vocabularies", default=True, action="store_false",
                        help="Whether to use previously saved vocabularies.")

    # Training and learning arguments
    parser.add_argument("--training_batch_size", type=int, default=50)
    parser.add_argument("--k", type=int, default=0, help="How many examples from the few-shot split to move to train.")
    parser.add_argument("--test_batch_size", type=int, default=1, help="Currently only 1 supported due to decoder.")
    parser.add_argument("--max_training_examples", type=int, default=None, help="If None all are used.")
    parser.add_argument("--learning_rate", type=float, default=0.001)
    parser.add_argument('--lr_decay', type=float, default=0.9)
    parser.add_argument('--lr_decay_steps', type=float, default=20000)
    parser.add_argument("--adam_beta_1", type=float, default=0.9)
    parser.add_argument("--adam_beta_2", type=float, default=0.999)
    parser.add_argument("--print_every", type=int, default=100)
    parser.add_argument("--evaluate_every", type=int, default=1000, help="How often to evaluate the model by decoding the "
                                                                         "test set (without teacher forcing).")
    parser.add_argument("--max_training_iterations", type=int, default=100000)
    parser.add_argument("--weight_target_loss", type=float, default=0.3, help="Only used if --auxiliary_task set.")

    # Testing and predicting arguments
    parser.add_argument("--max_testing_examples", type=int, default=None)
    parser.add_argument("--splits", type=str, default="test", help="comma-separated list of splits to predict for.")
    parser.add_argument("--max_decoding_steps", type=int, default=30, help="After 30 decoding steps, the decoding process "
                                                                           "is stopped regardless of whether an EOS token "
                                                                           "was generated.")
    parser.add_argument("--output_file_name", type=str, default="predict.json")

    # Situation Encoder arguments
    parser.add_argument("--simple_situation_representation", dest="simple_situation_representation", default=True,
                        action="store_true", help="Represent the situation with 1 vector per grid cell. "
                                                  "For more information, read grounded SCAN documentation.")
    parser.add_argument("--image_situation_representation", dest="simple_situation_representation", default=False,
                        action="store_false", help="Represent the situation with the full gridworld RGB image. "
                                                   "For more information, read grounded SCAN documentation.")
    parser.add_argument("--cnn_hidden_num_channels", type=int, default=50)
    parser.add_argument("--cnn_kernel_size", type=int, default=7, help="Size of the largest filter in the world state "
                                                                       "model.")
    parser.add_argument("--cnn_dropout_p", type=float, default=0.1, help="Dropout applied to the output features of the "
                                                                         "world state model.")
    parser.add_argument("--auxiliary_task", dest="auxiliary_task", default=False, action="store_true",
                        help="If set to true, the model predicts the target location from the joint attention over the "
                             "input instruction and world state.")
    parser.add_argument("--no_auxiliary_task", dest="auxiliary_task", default=True, action="store_false")

    # Command Encoder arguments
    parser.add_argument("--embedding_dimension", type=int, default=25)
    parser.add_argument("--num_encoder_layers", type=int, default=1)
    parser.add_argument("--encoder_hidden_size", type=int, default=100)
    parser.add_argument("--encoder_dropout_p", type=float, default=0.3, help="Dropout on instruction embeddings and LSTM.")
    parser.add_argument("--encoder_bidirectional", dest="encoder_bidirectional", default=True, action="store_true")
    parser.add_argument("--encoder_unidirectional", dest="encoder_bidirectional", default=False, action="store_false")

    # Decoder arguments
    parser.add_argument("--num_decoder_layers", type=int, default=1)
    parser.add_argument("--attention_type", type=str, default='bahdanau', choices=['bahdanau', 'luong'],
                        help="Luong not properly implemented.")
    parser.add_argument("--decoder_dropout_p", type=float, default=0.3, help="Dropout on decoder embedding and LSTM.")
    parser.add_argument("--decoder_hidden_size", type=int, default=100)
    parser.add_argument("--conditional_attention", dest="conditional_attention", default=True, action="store_true",
                        help="If set to true joint attention over the world state conditioned on the input instruction is"
                             " used.")
    parser.add_argument("--no_conditional_attention", dest="conditional_attention", default=False, action="store_false")

    # Other arguments
    parser.add_argument("--seed", type=int, default=42)
    
    
    
    try:
        get_ipython().run_line_magic('matplotlib', 'inline')
        args = parser.parse_args([])
    except:
        args = parser.parse_args()
    return args

In [81]:
if __name__ == "__main__":
    
    # Loading arguments
    args = arg_parse()
    try:
        get_ipython().run_line_magic('matplotlib', 'inline')
        # Experiment management:
        
        is_jupyter = True
    except:
        is_jupyter = False

usage: ipykernel_launcher.py [-h] --mode MODE
                             [--output_directory OUTPUT_DIRECTORY]
                             [--resume_from_file RESUME_FROM_FILE]
                             [--split SPLIT] [--data_directory DATA_DIRECTORY]
                             [--input_vocab_path INPUT_VOCAB_PATH]
                             [--target_vocab_path TARGET_VOCAB_PATH]
                             [--generate_vocabularies] [--load_vocabularies]
                             [--training_batch_size TRAINING_BATCH_SIZE]
                             [--k K] [--test_batch_size TEST_BATCH_SIZE]
                             [--max_training_examples MAX_TRAINING_EXAMPLES]
                             [--learning_rate LEARNING_RATE]
                             [--lr_decay LR_DECAY]
                             [--lr_decay_steps LR_DECAY_STEPS]
                             [--adam_beta_1 ADAM_BETA_1]
                             [--adam_beta_2 ADAM_BETA_2]
                

Traceback (most recent call last):
  File "<ipython-input-80-1381d86c6516>", line 110, in arg_parse
    args = parser.parse_args([])
  File "/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/argparse.py", line 1755, in parse_args
    args, argv = self.parse_known_args(args, namespace)
  File "/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/argparse.py", line 1787, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/argparse.py", line 2022, in _parse_known_args
    ', '.join(required_actions))
  File "/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/argparse.py", line 2508, in error
    self.exit(2, _('%(prog)s: error: %(message)s\n') % args)
  File "/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/argparse.py", line 2495, in exit
    _sys.exit(status)
SystemExit: 2

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

TypeError: object of type 'NoneType' has no len()

In [ ]:
def main(flags):
    for argument, value in flags.items():
        logger.info("{}: {}".format(argument, value))

    if not os.path.exists(flags["output_directory"]):
        os.mkdir(os.path.join(os.getcwd(), flags["output_directory"]))

    if not flags["simple_situation_representation"]:
        raise NotImplementedError("Full RGB input image not implemented. Implement or set "
                                  "--simple_situation_representation")
    # Some checks on the flags
    if flags["generate_vocabularies"]:
        assert flags["input_vocab_path"] and flags["target_vocab_path"], "Please specify paths to vocabularies to save."

    if flags["test_batch_size"] > 1:
        raise NotImplementedError("Test batch size larger than 1 not implemented.")

    data_path = os.path.join(flags["data_directory"], "dataset.txt")
    if flags["mode"] == "train":
        train(data_path=data_path, **flags)
    elif flags["mode"] == "test":
        assert os.path.exists(os.path.join(flags["data_directory"], flags["input_vocab_path"])) and os.path.exists(
            os.path.join(flags["data_directory"], flags["target_vocab_path"])), \
            "No vocabs found at {} and {}".format(flags["input_vocab_path"], flags["target_vocab_path"])
        splits = flags["splits"].split(",")
        for split in splits:
            logger.info("Loading {} dataset split...".format(split))
            test_set = GroundedScanDataset(data_path, flags["data_directory"], split=split,
                                           input_vocabulary_file=flags["input_vocab_path"],
                                           target_vocabulary_file=flags["target_vocab_path"], generate_vocabulary=False,
                                           k=flags["k"])
            test_set.read_dataset(max_examples=None,
                                  simple_situation_representation=flags["simple_situation_representation"])
            logger.info("Done Loading {} dataset split.".format(flags["split"]))
            logger.info("  Loaded {} examples.".format(test_set.num_examples))
            logger.info("  Input vocabulary size: {}".format(test_set.input_vocabulary_size))
            logger.info("  Most common input words: {}".format(test_set.input_vocabulary.most_common(5)))
            logger.info("  Output vocabulary size: {}".format(test_set.target_vocabulary_size))
            logger.info("  Most common target words: {}".format(test_set.target_vocabulary.most_common(5)))

            model = Model(input_vocabulary_size=test_set.input_vocabulary_size,
                          target_vocabulary_size=test_set.target_vocabulary_size,
                          num_cnn_channels=test_set.image_channels,
                          input_padding_idx=test_set.input_vocabulary.pad_idx,
                          target_pad_idx=test_set.target_vocabulary.pad_idx,
                          target_eos_idx=test_set.target_vocabulary.eos_idx,
                          **flags)
            model = model.cuda() if use_cuda else model

            # Load model and vocabularies if resuming.
            assert os.path.isfile(flags["resume_from_file"]), "No checkpoint found at {}".format(flags["resume_from_file"])
            logger.info("Loading checkpoint from file at '{}'".format(flags["resume_from_file"]))
            model.load_model(flags["resume_from_file"])
            start_iteration = model.trained_iterations
            logger.info("Loaded checkpoint '{}' (iter {})".format(flags["resume_from_file"], start_iteration))
            output_file_name = "_".join([split, flags["output_file_name"]])
            output_file_path = os.path.join(flags["output_directory"], output_file_name)
            output_file = predict_and_save(dataset=test_set, model=model, output_file_path=output_file_path, **flags)
            logger.info("Saved predictions to {}".format(output_file))
    elif flags["mode"] == "predict":
        raise NotImplementedError()
    else:
        raise ValueError("Wrong value for parameters --mode ({}).".format(flags["mode"]))


if __name__ == "__main__":
    input_flags = vars(parser.parse_args())
    main(flags=input_flags)